<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Manipulating-data" data-toc-modified-id="Manipulating-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Manipulating data</a></span></li><li><span><a href="#Machine-learning-pipelines" data-toc-modified-id="Machine-learning-pipelines-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Machine learning pipelines</a></span></li><li><span><a href="#Model-tuning-and-selection" data-toc-modified-id="Model-tuning-and-selection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model tuning and selection</a></span></li></ul></div>

# Intro

* how to read files
* write tables 
* manage data

* first step is to connect to a Spark cluster from PySpark by creating an instance of 'SparkContext' class
* second to work with Spark DataFrames create 'SparkSession'(think of it as your interface with that connection) object from SparkContext (think of this as connection to clusters of computers) 

In [37]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
from pyspark.sql import SparkSession #import SparkSession 

my_spark = SparkSession.builder.getOrCreate() # create new sparksession
my_spark

In [4]:
# look up all the tables present in the cluster/catalog
# spark.catalog.listTables()
flights = my_spark.read.csv('flights_small.csv')
flights.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
| _c0|  _c1|_c2|     _c3|      _c4|     _c5|      _c6|    _c7|    _c8|   _c9|  _c10|_c11|    _c12|    _c13|_c14|  _c15|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
+----+-----+---+--------+---------+-----

In [5]:
flights.describe().show()

+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-------+-----------------+------+-----+------------------+-----------------+------------------+-----------------+
|summary|   _c0|               _c1|              _c2|               _c3|               _c4|               _c5|               _c6|    _c7|    _c8|              _c9|  _c10| _c11|              _c12|             _c13|              _c14|             _c15|
+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-------+-----------------+------+-----+------------------+-----------------+------------------+-----------------+
|  count| 10001|             10001|            10001|             10001|             10001|             10001|             10001|  10001|  10001|            10001| 10001|10001|             10001|            10001|             10001|            100

In [6]:
# run sql queries
# Register the DataFrame as a SQL temporary view
flights.createOrReplaceTempView("flight")
flights5 = my_spark.sql("SELECT * FROM flight LIMIT 5")
flights5.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
| _c0|  _c1|_c2|     _c3|      _c4|     _c5|      _c6|    _c7|    _c8|   _c9|  _c10|_c11|    _c12|    _c13|_c14|  _c15|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
+----+-----+---+--------+---------+-----

In [7]:
flights.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15']

In [8]:
# convert it to pandas dataframe
df = flights.toPandas()
df.head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15
0,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
1,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58
2,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40
3,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111,679,14,43
4,2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83,569,17,5


In [9]:
header = df.iloc[0]
df = df[1:]
df1 = df.rename(columns = header)

In [10]:
df1.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
1,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58
2,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40
3,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111,679,14,43
4,2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83,569,17,5
5,2014,3,9,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127,937,7,54


In [11]:
df1['distance'] = df1['distance'].astype(int)
#df1['air_time'] = df1['air_time'].astype(int)
df1.dtypes

year         object
month        object
day          object
dep_time     object
dep_delay    object
arr_time     object
arr_delay    object
carrier      object
tailnum      object
flight       object
origin       object
dest         object
air_time     object
distance      int64
hour         object
minute       object
dtype: object

In [12]:
# pandas to spark
import pandas as pd
import numpy as np
pd_temp = pd.DataFrame(np.random.random(10))
spark_temp = my_spark.createDataFrame(pd_temp) # pandas to spark
print(spark_temp.show())
spark_temp.createOrReplaceTempView('temp') #adding spark_temp to catalog

+-------------------+
|                  0|
+-------------------+
| 0.6805605795545658|
| 0.5796672445546437|
|0.18098062759131417|
|0.35693559899055505|
| 0.8229209794315293|
|0.41676013375254084|
|0.49450912512842615|
|0.09724552280688625|
|  0.620870536746868|
| 0.3120816713320692|
+-------------------+

None


In [13]:
spark.catalog.listTables()

[Table(name='flight', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [14]:
# read url 
planes1 = pd.read_csv('https://assets.datacamp.com/production/course_4452/datasets/planes.csv')
#convert to spark
planes = my_spark.createDataFrame(planes1)
planes.show(2)
#add to catalog
planes.createOrReplaceTempView('planes')
my_spark.catalog.listTables()

+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|  year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
| N103US|1999.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
only showing top 2 rows



[Table(name='flight', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='planes', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

# Manipulating data

* learn about pyspark.sql.module - provides optimized data queries

In [15]:
flights = my_spark.createDataFrame(df1)
flights.createOrReplaceTempView('flights')
flights = spark.table('flights')
flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [16]:
my_spark.catalog.listTables()

[Table(name='flight', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='planes', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [17]:
# add new col- duration_hrs
flights = flights.withColumn('duration_hrs',flights['air_time']/60)

In [18]:
flights.select('duration_hrs').show()

+------------------+
|      duration_hrs|
+------------------+
|               2.2|
|               6.0|
|              1.85|
|1.3833333333333333|
|2.1166666666666667|
|2.0166666666666666|
|               1.5|
|1.6333333333333333|
|              2.25|
|               3.3|
|2.1666666666666665|
| 2.566666666666667|
|2.1166666666666667|
|              3.05|
|              2.15|
|               1.5|
|1.2666666666666666|
|               3.6|
| 4.833333333333333|
|              1.85|
+------------------+
only showing top 20 rows



In [19]:
my_spark.sql('SELECT origin, dest, COUNT(*) FROM flights \
GROUP BY origin, dest').show(2)

+------+----+--------+
|origin|dest|count(1)|
+------+----+--------+
|   SEA| RNO|       8|
|   SEA| DTW|      98|
+------+----+--------+
only showing top 2 rows



In [20]:
my_spark.sql('SELECT AVG(air_time)/60 FROM flights \
GROUP BY origin, carrier').show()

+----------------------------------------------------+
|(avg(CAST(air_time AS DOUBLE)) / CAST(60 AS DOUBLE))|
+----------------------------------------------------+
|                                   2.641062801932367|
|                                  3.4815763052208837|
|                                  2.5649012189995797|
|                                  2.9572755417956658|
|                                  1.5976558033161805|
|                                  3.4336904761904763|
|                                   1.293419540229885|
|                                    2.78377065111759|
|                                  3.3224926253687315|
|                                  2.1333333333333333|
|                                  3.2710648148148147|
|                                   3.027322796934866|
|                                  1.4189814814814814|
|                                   3.008447488584475|
|                                  1.8805555555555555|
|         

In [21]:
# filter which is same as WHERE in SQL
long_flights1 = flights.filter('distance > 1000')
long_flights12 = flights.filter(flights.distance > 1000) # same as above
long_flights1.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|        2.25|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 2 rows



In [22]:
# select cols
selected1 = flights.select('tailnum','origin','dest')
selected1.show(2)
temp = flights.select('origin', 'dest', 'carrier')
temp.show(2)
filterA = flights.origin == 'SEA'
filterB = flights.dest == 'LAX'
temp.filter(filterA).show(2)

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
+-------+------+----+
only showing top 2 rows

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| LAX|     VX|
|   SEA| HNL|     AS|
+------+----+-------+
only showing top 2 rows

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| LAX|     VX|
|   SEA| HNL|     AS|
+------+----+-------+
only showing top 2 rows



In [23]:
# use of .alias() and .selectExpr() for same task
avg_speed = (flights.distance/(flights.air_time/60)).alias('avg_speed')
speed1 = flights.select('origin','dest','tailnum',avg_speed)
#above 2 tasks in one liner
speed2 = flights.selectExpr('origin','dest','tailnum',  
                            'distance/(air_time/60) as avg_speed')
speed1.show(2)
speed2.show(2)

+------+----+-------+-----------------+
|origin|dest|tailnum|        avg_speed|
+------+----+-------+-----------------+
|   SEA| LAX| N846VA|433.6363636363636|
|   SEA| HNL| N559AS|446.1666666666667|
+------+----+-------+-----------------+
only showing top 2 rows

+------+----+-------+-----------------+
|origin|dest|tailnum|        avg_speed|
+------+----+-------+-----------------+
|   SEA| LAX| N846VA|433.6363636363636|
|   SEA| HNL| N559AS|446.1666666666667|
+------+----+-------+-----------------+
only showing top 2 rows



In [24]:
# aggregating
flights.filter(flights.origin == 'PDX').groupBy().min('distance').show(2)
flights.filter(flights.origin == 'SEA').groupBy().max('duration_hrs').show(2)

+-------------+
|min(distance)|
+-------------+
|          106|
+-------------+

+-----------------+
|max(duration_hrs)|
+-----------------+
|6.816666666666666|
+-----------------+



In [25]:
flights.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: long (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)



In [26]:
# aggregating
flights.filter(flights.carrier == 'DL').filter(flights.origin == 'SEA').groupBy().avg('duration_hrs').show()
flights.withColumn('Total_hrs',flights.air_time/60).groupBy().sum('Total_hrs').show()

+------------------+
| avg(duration_hrs)|
+------------------+
|3.1367816091954035|
+------------------+

+-----------------+
|   sum(Total_hrs)|
+-----------------+
|25289.60000000004|
+-----------------+



In [27]:
my_spark.sql('SELECT origin, dest FROM flights WHERE carrier == "DL"').show(2)

+------+----+
|origin|dest|
+------+----+
|   SEA| LAX|
|   SEA| ATL|
+------+----+
only showing top 2 rows



In [28]:
# aggregating + groupby
flights.groupBy('tailnum').count().show(2)
flights.groupBy('origin').avg('duration_hrs').show()


+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
+-------+-----+
only showing top 2 rows

+------+------------------+
|origin| avg(duration_hrs)|
+------+------------------+
|   SEA|2.6739358267520963|
|   PDX| 2.285257208048123|
+------+------------------+



In [29]:
# change type
from pyspark.sql.types import IntegerType
flights = flights.withColumn("dep_delay", flights["dep_delay"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))

In [30]:
import pyspark.sql.functions as F
by_month_dest = flights.groupBy('month','dest')
by_month_dest.avg('dep_delay').show(2)
by_month_dest.agg(F.stddev('dep_delay')).show(2)

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|   11| TUS|-2.3333333333333335|
|   11| ANC|  7.529411764705882|
+-----+----+-------------------+
only showing top 2 rows

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|   11| TUS|     3.055050463303893|
|   11| ANC|    18.604716401245316|
+-----+----+----------------------+
only showing top 2 rows



In [31]:
#read in file in spark -> convert to pandas 
# rename col name -> convert to spark
# add to catalog -> read from the catalog
airpots = my_spark.read.csv('airports.csv')
df2 = airpots.toPandas()
header = df2.iloc[0]
df2 = df2[1:]
df3 = df2.rename(columns= header)
airpot = my_spark.createDataFrame(df3)
airpot.createOrReplaceTempView('airpots')
my_spark.catalog.listTables()
airpots = my_spark.table('airpots')
airpots.show(2)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 2 rows



In [32]:
# rename first col
airpots =airpots.withColumnRenamed('faa','dest')
flights_with_airports = flights.join(airpots,'dest','leftouter')
flights_with_airports.show(2)

+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+----------+---+---+---+
|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|air_time|distance|hour|minute|      duration_hrs|                name|      lat|       lon|alt| tz|dst|
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+----------+---+---+---+
| MSY|2014|    8| 16|    1014|       -1|    1644|      -16|     AS| N549AS|   788|   SEA|     248|    2086|  10|    14| 4.133333333333334|Louis Armstrong N...|29.993389|-90.258028|  4| -6|  A|
| MSY|2014|   10| 29|    1009|       -6|    1613|      -47|     AS| N532AS|   788|   SEA|     224|    2086|  10|     9|3.7333333333333334|Louis Armstrong N...|29.993389|-90.258028|  4| -6|  A|
+----+----+-----+---+--------+-----

# Machine learning pipelines

In [38]:
planes.show(2)
flights.show(2)

+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|  year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
| N103US|1999.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
only showing top 2 rows

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|   12|

In [73]:
planes = planes.withColumnRenamed('year','plane_year') #rename a col
model_data = flights.join(planes,'tailnum','leftouter')
# need arr_delay, air_time, month, plane_year, plane_age as inputs to our model
model_data = model_data.withColumn('arr_delay',
                                   model_data.arr_delay.cast('integer'))
model_data = model_data.withColumn('month',
                                   model_data.month.cast('integer'))
model_data = model_data.withColumn('plane_age',
                                   model_data.year - model_data.plane_year)#extract year 
model_data = model_data.withColumn('plane_year',
                                   model_data.plane_year.cast('integer'))
model_data = model_data.withColumn('plane_age',
                                   model_data.plane_age.cast('integer'))
model_data.printSchema()

root
 |-- tailnum: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: long (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)
 |-- plane_year: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: long (nullable = true)
 |-- seats: long (nullable = true)
 |-- speed: double (nullable = true)
 |-- engine: string (nullable = true)
 |-- plane_age: integer (nul

In [74]:
#output- plane delayed or not
model_data = model_data.withColumn('is_late',model_data.arr_delay > 0)
model_data = model_data.withColumn("label", model_data.is_late.cast('integer'))
model_data = model_data.filter('arr_delay is not NULL and air_time is not NULL and dep_delay is not NULL and plane_year is not NULL') # remove missing values from those cols

In [75]:
# convert strings using one-hot-vectors (2 step process). we need carrier, dest as inputs
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

carr_index = StringIndexer(inputCol='carrier',outputCol='carrier_index')
carr_encoder = OneHotEncoder(inputCol='carrier_index',outputCol='carrier_fact')

dest_indexer = StringIndexer(inputCol='dest',outputCol='dest_index')
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

#make a VectorAssembler- where all of the cols are combined into a single col
vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact", "plane_age"], outputCol='features')

# create the pipeline- lets you reuse the same modeling process over and over again by wrapping it up in one simple object.
from pyspark.ml import Pipeline

#stages should be a list holding all the stages you want your data to go through in the pipeline. 
flights_pipe = Pipeline(stages = [dest_indexer,dest_encoder,carr_index,carr_encoder,vec_assembler])

In [76]:
# transform the data through pipeline
# Fit and transform the data
piped_data = flights_pipe.fit(model_data).transform(model_data)


In [77]:
# split the data
training, test = piped_data.randomSplit([0.6,0.4]) #60% training, 40% test

# Model tuning and selection

* logistic regression will be used- predicts probability
* To use this as a classification algorithm, all you have to do is assign a cutoff point to these probabilities. If the predicted probability is above the cutoff point, you classify that observation as a 'yes' (in this case, the flight being late), if it's below, you classify it as a 'no'!
* fine-tune this model by testing different values for several hyperparameters. A hyperparameter is just a value in the model that's not estimated from the data, but rather is supplied by the user to maximize performance.
* fine-tune using k-fold cross validation. The cross validation error is an estimate of the model's error on the test set.

In [78]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pyspark.ml.tuning as tune

In [79]:
lr = LogisticRegression() 
evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC') #compare models using the metric output of the evaluator

#make a grid
grid = tune.ParamGridBuilder()
grid = grid.addGrid(lr.regParam, np.arange(0,0.1,0.01)) #hyperparameters
grid = grid.addGrid(lr.elasticNetParam,[0,1]) # another grid
grid = grid.build() #build the grid

#make the validator
cv = tune.CrossValidator(estimator=lr,estimatorParamMaps=grid,
                        evaluator=evaluator)

In [80]:
# Fit cross validation models
#models = cv.fit(training)
# Extract the best model
#best_lr = models.bestModel

#fit models
best_lr = lr.fit(training)

* common metric for binary classification algorithms call the AUC

In [82]:
# model evaluation on test data
test_results = best_lr.transform(test)
#prediction
evaluator.evaluate(test_results)

0.6994497868834203

ROC: 70% or 0.7 is good! The closer the value is to 1 the better the model